In [42]:
using Pkg
Pkg.activate("../.")
using Revise
using NotebookToLatex
# import NotebookToLatex:createproject, insertlinebelow
using JSON
using ReadableRegex
# using Base64
using Librsvg_jll

  Activating environment at `~/MEGA/EMAp/NotebookToLatex.jl/Project.toml`


In [6]:
notebook = read("jupyternotebook.ipynb", String);

In [55]:
jsonnb = JSON.parse(notebook)

Dict{String, Any} with 4 entries:
  "cells"          => Any[Dict{String, Any}("cell_type"=>"markdown", "source"=>…
  "nbformat_minor" => 5
  "metadata"       => Dict{String, Any}("language_info"=>Dict{String, Any}("fil…
  "nbformat"       => 4

In [73]:
join(jsonnb["cells"][8]["outputs"][1]["text"])

"0.17603266338214976\n0.12098536225957168\n0.06475879783294587\n0.02699548325659403\n0.00876415024678427\n0.0022159242059690038\n0.0004363413475228801\n6.691511288244268e-5\n7.991870553452737e-6\n7.433597573671488e-7\n"

In [8]:
md = join(jsonnb["cells"][5]["source"])

"The Wasserstein Distance for 1D distributions can be obtained by:\n\$\$\n\\int_0^1 |C_\\alpha^{-1}(r) - C_\\beta^{-1}(r)|^p dr\n\$\$\n\nWhere \$C_\\alpha^{-1}\$ is the quantile function for the distribution \$\\alpha\$ (the inverse of the Cumulative Distribution Function).\n\nWrite some more markdown, with **bold text here** and *italics* and\n[a link here](https://juliaoptimaltransport.github.io/OptimalTransport.jl/stable/examples/basic/),\nanother **bold** and *more italics* and\n[another](https://github.com/). Ends with two figures\n![Figure](figure.svg) and\n![Figure 2](figure.svg) again."

In [79]:
jupytertolatex("jupyternotebook.ipynb")